In [4]:
import numpy as np
from pyscf import fci
from copy import deepcopy, copy

#define vacuum
#take in operator string
#format: string of indices, bitstring of creation vs annihilation
#return normal-ordered operator string plus contracted terms, according to Wick's theorem



In [5]:
norbs = 3
Nalpha = 2
vacuumOccupations = [1 for i in range(Nalpha)] + [0 for i in range (norbs - Nalpha)]
print(vacuumOccupations)

[1, 1, 0]


In [76]:
class basicOperator:
    def __init__(self, orbital_, creation_annihilation_, spin_):
        self.orbital = orbital_ #orbital index
        self.spin = spin_ #1 for alpha, 0 for beta
        self.creation_annihilation = creation_annihilation_ #1 for creation, 0 for annihilation
#        self.quasi_cre_ann = None

    def __str__(self):
        string = "a"
        if bool(self.creation_annihilation):
            string = string + "^"
        else:
            string = string + "_"
        if bool(self.spin):
            string = string + "{" + str(self.orbital) + "\\alpha}"
        else:
            string = string + "{" + str(self.orbital) + "\\beta}"
        return string

    def apply(self, state):
        neleca, nelecb = state.nelec[0], state.nelec[1]
        norb = state.norb
        if bool(self.creation_annihilation):
            if bool(self.spin):
#                if neleca == norb:
#                    state.array = np.array([[0.]])
#                    state.nelec = (0, 0)
#                else:
                state.array = fci.addons.cre_a(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca + 1, nelecb)
            else:
#                if nelecb == norb:
#                    state.array = np.array([[0.]])
#                    state.nelec = (0, 0)
#                else:
                state.array = fci.addons.cre_b(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca, nelecb + 1)
        else:
            if bool(self.spin):
#                if neleca == 0:
#                    state.array = np.array([[0.]])
#                    state.nelec = (0, 0)
#                else:
                state.array = fci.addons.des_a(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca - 1, nelecb)
            else:
#                if nelecb == 0:
#                    state.array = np.array([[0.]])
#                    state.nelec = (0, 0)
#                else:
                state.array = fci.addons.des_b(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca, nelecb - 1)

In [77]:
class operatorSum:
    def __init__(self, operatorList_):
        self.operatorList = operatorList_

    def apply(self, state_):
        result = state(np.array([[0.]]), state_.norb, state_.nelec)
        for o in self.operatorList:
            statecopy = copy(state_)
            o.apply(statecopy)
            print(statecopy.array)
            result = statecopy + result
            print(result.array)
#        state1, state2 = copy(state), copy(state)
#        self.operator1.apply(state1)
#        self.operator2.apply(state2)
#        state = state1 + state2
        state_ = copy(result)
        print(state_.array)

In [78]:
class operatorProduct:
    def __init__(self, operatorList_):
        self.operatorList = operatorList_

    def apply(self, state):
        i = len(self.operatorList)
        while i>0:
            i = i - 1
            self.operatorList[i].apply(state)

In [79]:
def normalOrder(operatorProduct_, vacuum):
    '''
    Input: an operatorProduct and a list corresponding to which orbitals are occupied in the Fermi vacuum
    Output: operatorProduct_ normal ordered with respect to vacuum
    '''
    quasiCreationList, quasiAnnihilationList = [], []
    for o in range(len(operatorProduct_.operatorList)):
        operator = operatorProduct_.operatorList[o]
        print(operator.orbital)
        print(operator.creation_annihilation)
        print(operator.spin)
        if bool(operator.creation_annihilation):
            if bool(vacuum[operator.orbital]):
                quasiAnnihilationList.append(operator)
            else:
                quasiCreationList.append(operator)
        else:
            if bool(vacuum[operator.orbital]):
                quasiCreationList.append(operator)
            else:
                quasiAnnihilationList.append(operator)
    return operatorProduct(quasiCreationList + quasiAnnihilationList)

In [84]:
oP = operatorProduct([basicOperator(1,1,1), basicOperator(2,1,1)])
nOP = normalOrder(oP, vacuumOccupations)
for i in range(len(nOP.operatorList)):
    print(nOP.operatorList[i])

1
1
1
2
1
1
a^{2\alpha}
a^{1\alpha}


In [11]:
#Normal-ordered particle number-conserving excitation operators
class excitation(operatorProduct):
    def __init__(self, creationList_, annihilationList_, spinList_):
        self.operatorList = []
        for i in range(len(creationList_)):
            self.operatorList.append(basicOperator(creationList_[i], 1, spinList_[i]))
        for i in range(len(annihilationList_)):
            self.operatorList.append(basicOperator(annihilationList_[i], 0, spinList_[i+len(creationList_)]))

In [12]:
class spinFreeExcitation(operatorSum):
    def __init__(self, creationList_, annihilationList_):
        self.operatorList = []
        spinLists = np.reshape(np.zeros(len(creationList_)), (1, -1))
        for i in range(len(creationList_)):
            newspinLists = spinLists.copy()
            for s in range(len(spinLists)):
                newspinLists[s,i] = 1
            spinLists = np.concatenate((spinLists, newspinLists))
        for l in range(len(spinLists)):
            spinList = spinLists[l]
            self.operatorList.append(excitation(creationList_, annihilationList_, [*spinList, *spinList[::-1]]))

In [13]:
class operatorString:
    def __init__(self, string_):
        self.string = string_
        #list of indices in the string of second-quantised operators
        self.iS = []
        #bitstring of same length, with 1 corresponding to creation and 0 for annihilation
        self.cOA = []
        #bitstring of same length, with 1 corresponding to alpha and 0 for beta
        self.aOB = []

    #Get indices, alpha or beta, and creation or annihilation from string
    def parse(self):
        c = 0
        while c < len(self.string) - 1:
            self.iS.append(int(self.string[c]))
            if self.string[c+1] == "a":
                    self.aOB.append(1)
            elif self.string[c+1] == "b":
                    self.aOB.append(0)
            if c < len(self.string) - 2 and self.string[c+2] == "+":
                self.cOA.append(1)
                c = c + 3
            else:
                self.cOA.append(0)
                c = c + 2

    #Change string based on new indices, alpha or beta, and creation or annihilation
    def updateString(self):
        self.string = ""
        i = 0
        while i < len(self.iS):
            if bool(self.cOA[i]):
                if bool(self.aOB[i]):
                    self.string = self.string + str(self.iS[i]) + "a+"
                else:
                    self.string = self.string + str(self.iS[i]) + "b+"
            else:
                if bool(self.aOB[i]):
                    self.string = self.string + str(self.iS[i]) + "a"
                else:
                    self.string = self.string + str(self.iS[i]) + "b"
            i = i + 1

    #Apply the string of second-quantised operators to an arbitrary state object
    #Note that nelec does not yet account for the state being completely annihilated
    def apply(self, state):
        i = len(self.iS)
        while(i > 0):
            i = i - 1
#            print(i)
#            print(self.iS[i])
#            print(self.cOA[i])
#            print(self.aOB[i])
            neleca, nelecb = state.nelec[0], state.nelec[1]
            if bool(self.cOA[i]):
                if bool(self.aOB[i]):
                    state.array = fci.addons.cre_a(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca + 1, nelecb)
                else:
                    state.array = fci.addons.cre_b(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca, nelecb + 1)
            else:
                if bool(self.aOB[i]):
                    state.array = fci.addons.des_a(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca - 1, nelecb)
                else:
                    state.array = fci.addons.des_b(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca, nelecb - 1)

In [14]:
class state:
    def __init__(self, array_, norb_, nelec_):
        self.array = array_
        self.norb = norb_
        self.nelec = nelec_

    def __add__(self, other):
        if (other.array == 0.).all():
            return self
        elif (self.array == 0.).all():
            return other
        elif self.norb == other.norb and self.nelec == other.nelec:
            newArray = self.array + other.array
            return state(newArray, self.norb, self.nelec)
        else:
            print("Adding states of different orbital or particle numbers or different spin projections is not supported. Returning original state")
            return self

    def __radd__(self, other):
        if other == 0:
            return self
        else:
            return self.__add__(other)


In [15]:
ref = state(np.array([[1.,0.,0.],[0.,0.,0.],[0.,0.,0.]]), 3, (2,1))

In [16]:
new = copy(ref)

In [17]:
des0a, des0b = basicOperator(0,0,1), basicOperator(0,0,0)
des1a, des1b = basicOperator(1,0,1), basicOperator(1,0,0)
des2a, des2b = basicOperator(2,0,1), basicOperator(2,0,0)

cre0a, cre0b = basicOperator(0,1,1), basicOperator(0,1,0)
cre1a, cre1b = basicOperator(1,1,1), basicOperator(1,1,0)
cre2a, cre2b = basicOperator(2,1,1), basicOperator(2,1,0)

In [18]:
exc12a = operatorProduct([cre2a, des1a])
exc12b = operatorProduct([cre2b, des1b])
exc12 = operatorSum([exc12a, exc12b])

In [19]:
new.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [20]:
exc12.apply(new)

[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]


In [21]:
new.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [22]:
new.nelec

(2, 1)

In [23]:
opStr = operatorString("0a1b+2a+")
opStr.parse()

In [24]:
opStr.iS

[0, 1, 2]

In [25]:
new = deepcopy(ref)
opStr.apply(new)

In [26]:
ref.nelec

(2, 1)

In [27]:
new.nelec

(2, 2)

In [28]:
new.array

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [-1.,  0.,  0.]])

In [29]:
opStr1 = operatorString("0a+0b+1a+")
opStr1.parse()
print(opStr1.string)
print(opStr1.iS)
print(opStr1.aOB)
print(opStr1.cOA)

0a+0b+1a+
[0, 0, 1]
[1, 0, 1]
[1, 1, 1]


In [30]:
opStr2 = copy(opStr1)

In [31]:
type(opStr2.string)

str

In [32]:
opStr2.iS[2] = 2
print(opStr2.string)
print(opStr2.iS)
print(opStr2.aOB)
print(opStr2.cOA)

0a+0b+1a+
[0, 0, 2]
[1, 0, 1]
[1, 1, 1]


In [33]:
opStr2.updateString()

In [34]:
opStr2.string

'0a+0b+2a+'

In [35]:
ref.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [36]:
new = copy(ref)

In [37]:
new.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [38]:
opStr.apply(new)

In [39]:
new.array

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [-1.,  0.,  0.]])

In [40]:
cre0a = basicOperator(0,1,1)

In [41]:
cre0a.apply(new)

In [42]:
new.array

array([[-1.,  0.,  0.]])

In [43]:
new.nelec

(3, 2)

In [44]:
des1 = operatorSum([basicOperator(1,1,0), basicOperator(1,0,0)])

In [45]:
des1.apply(new)

[[0.]]
[[0.]]
[[1. 0. 0.]]
[[1. 0. 0.]]
[[1. 0. 0.]]


In [46]:
E12 = spinFreeExcitation([2],[1])

In [47]:
E12.apply(new)

[[ 0. -1. -0.]]
[[ 0. -1. -0.]]
[[0. 0. 0.]]
[[ 0. -1. -0.]]
[[ 0. -1. -0.]]


In [48]:
new.array

array([[-1.,  0.,  0.]])

In [49]:
des1a.apply(new)

In [50]:
new.nelec

(2, 2)

In [51]:
new.array

array([[ 0.,  0.,  0.],
       [ 1., -0., -0.],
       [ 0.,  0.,  0.]])

In [52]:
ref.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [53]:
ref.nelec

(2, 1)

In [54]:
new = copy(ref)

In [55]:
des1b.apply(new)

In [56]:
new.array

array([[0.],
       [0.],
       [0.]])

In [57]:
new.nelec

(2, 0)

In [58]:
type(new)

__main__.state

In [59]:
ref.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])